<a href="https://colab.research.google.com/github/otavioeliass/F1_project/blob/main/F1_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import glob
import re

#1) LOADING RACE OVERALL DATA
First we start by loading the overall race results from all years (1950 - 2021). We then save this dataframe into a .csv file for future use with easier loading process.


In [ ]:
# -------------- Loading overall race results from 2010 until 2021 --------------
years = np.arange(1950,2022,1)
type = 'Race Results'
name = 'overall_race_results.csv'

overalls_list = []

# iterates through all years file
for year in years: 
  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/F1_dataset/'+str(year)+'/'+type+'/'+name, header=0) # get the overall_race_results.csv inside each year file
  overalls_list.append(df)

# concatenate all dataframes
df_overalls = pd.concat(overalls_list, axis=0, ignore_index=True) 

# Team name standardization using exteranal file
team_names_fix = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/F1_dataset/team_names_fix.xlsx', index_col=False)

for i, row in team_names_fix.iterrows():
  df_overalls.replace(row['Team'], row['New'], inplace=True)

# write the consolidated .csv file
df_overalls.to_csv('/content/drive/MyDrive/Colab Notebooks/F1_dataset/'+'F1_overalls_dataset.csv', index=False) 

#df_overalls.head()

# 2) LOADING RACE DETAILED DATA
The second step is to load the race detailed results from all years (1950 - 2021). We then save this dataframe into a .csv file for future use with easier loading process.

In [ ]:
# -------------- Loading every race data from year 2010 until 2021 --------------
# Adding year and race name info to the loaded data set as new columns in the df
years = np.arange(1950,2022,1)
type = 'Race Results'

path = r'/content/drive/MyDrive/Colab Notebooks/F1_dataset/'


df_list = []

for year in years: # iterates over the years folders
  all_files = glob.glob(path+str(year)+'/'+type+'/*.csv') # list all files in a given folder which name's end with ".csv"
  for filename in all_files: # iterates over all .csv files of a given folder
    if filename.endswith('/overall_race_results.csv'): # ignore the overall race results file
      continue
    df = pd.read_csv(filename, index_col=None, header=0)
    df['Year'] = year # add the year column to the dataframe
    race = re.findall(path+str(year)+'/'+type+'/(.*)_race_results.csv',filename) # retrieve the race name from the file name
    df['race_name'] = race[0] # add the race name column to the dataframe
    df['Driver Number'].astype('int64') # force all driver's number to integer
    df_list.append(df)

# concatenate all dataframes
df_race_results = pd.concat(df_list, axis=0, ignore_index=True) 

# Team name standardization using exteranal file
team_names_fix = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/F1_dataset/team_names_fix.xlsx', index_col=False)

for i, row in team_names_fix.iterrows():
  df_race_results.replace(row['Team'], row['New'], inplace=True)

# write the consolidated .csv file
df_race_results.to_csv(path+'F1_races_dataset.csv', index=False) 

# df_race_results.head()

# 3) LOADING RACE QUALIFYING DETAILED DATA
After that we load the race qualifying detailled results from all years (1950 - 2021). We then save this dataframe into a .csv file for future use with easier loading process.

In [ ]:
# -------------- Loading every race qualify data from year 2010 until 2021 --------------
# Adding year and race name info to the loaded data set as new columns in the df
years = np.arange(1950,2022,1)
type = 'Qualifying Results'
name = 'overall_race_results.csv'

path = r'/content/drive/MyDrive/Colab Notebooks/F1_dataset/'


df_list = []

# iterates over the years folders
for year in years: 
  all_files = glob.glob(path+str(year)+'/'+type+'/*.csv')
  # iterates over all .csv files of a given folder
  for filename in all_files: 
    df = pd.read_csv(filename, index_col=None, header=0)
    df['Year'] = year
    race = re.findall(path+str(year)+'/'+type+'/(.*)_qualifying_results.csv',filename)
    df['race_name'] = race[0]
    df['Driver Number'].astype('int64')
    df_list.append(df)

df_qualifying_results = pd.concat(df_list, axis=0, ignore_index=True)

# Team name standardization using external file
team_names_fix = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/F1_dataset/team_names_fix.xlsx', index_col=False)

for i, row in team_names_fix.iterrows():
  df_qualifying_results.replace(row['Team'], row['New'], inplace=True)

# write the consolidated .csv file
df_qualifying_results.to_csv(path+'F1_qualifying_dataset.csv', index=False) 

#df_qualifying_results.head()

# 3.1/2) Files fast loading

In [1]:
import pandas as pd
import numpy as np
import glob
import re

# Loading data set from output files of the above cells
df_overalls = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/F1_dataset/F1_overalls_dataset.csv', index_col=None, header=0)
df_race_results = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/F1_dataset/F1_races_dataset.csv', index_col=None, header=0)
df_qualifying_results = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/F1_dataset/F1_qualifying_dataset.csv', index_col=None, header=0)

# 4.1) Who are the most succesfull drivers and teams in F1 history?
Succes can be a relative concept, therefore, not so trivial to measure. A driver who manage to score points, even though he/she has the least competitive car on the grid can be considered succesfull. 

So in this instance, considering the information the dataset offer us, we will assume that championship titles and race wins are sinonyms of succes.

In order to answer these questions I'll evaluate the following statistics:

A) Most titles by driver

B) Most titles by team

C) Most titles/total seasons by driver

D) Most titles/total seasons  by team

E) Most race wins by driver

F) Most race wins by team

G) Most race wins/total races by driver

H) Most race wins/total races by team

I) Most podiums by driver

J) Most podiums by team

K) Most podiums/total races by driver

L) Most podiums/total races by team

## A) Most titles by driver

In [ ]:
# ----------------- Fixing some races points to generate a correct list of champions, due to points regulation change throughout the years -----------------
race_results_titles = df_race_results
r = (race_results_titles['Driver'] == 'Graham Hill') & (race_results_titles['Year'] == 1964) & (race_results_titles['race_name'] == 'belgium')
race_results_titles.loc[r,'Points'] = 0

r = (race_results_titles['Driver'] == 'Ayrton Senna') & (race_results_titles['Year'] == 1988) & (race_results_titles['race_name'].isin(['portugal','spain']))
race_results_titles.loc[r,'Points'] = 0

r = (race_results_titles['Driver'] == 'Alain Prost') & (race_results_titles['Year'] == 1988) & (race_results_titles['race_name'].isin(['hungary','japan','belgium']))
race_results_titles.loc[r,'Points'] = 0
# ----------------- end of points regulation fix -----------------

# Sumup the points by driver and by year
driver_champ = race_results_titles.groupby(by=['Year','Driver'])['Points'].sum() 

# cast the groupby result into a dataframe
driver_champ = pd.DataFrame(driver_champ) 
# sort the dataframe in descending order by year, then by points
driver_champ.sort_values(['Year','Points'], ascending=[False, False], inplace=True) 
driver_champ.reset_index(inplace=True)

driver_champ.groupby(by=['Year'])['Driver'].head(1).value_counts().head(10)

Lewis Hamilton        7
Michael Schumacher    7
Juan Manuel Fangio    5
Alain Prost           4
Sebastian Vettel      4
Jack Brabham          3
Niki Lauda            3
Nelson Piquet         3
Ayrton Senna          3
Jackie Stewart        3
Name: Driver, dtype: int64

## B) Most titles by team

In [ ]:
# ----------------- End of Team names standardization -----------------

# Sumup the points by team and by year
team_champ = race_results_titles.groupby(by=['Year','Car'])['Points'].sum() 
# cast the groupby result into a dataframe
team_champ = pd.DataFrame(team_champ) 
team_champ.reset_index(inplace=True)
# selecting only years after 1958 included, years in which the constructor championship was awarded
team_champ.drop(team_champ.loc[team_champ['Year'] < 1958].index, inplace=True) 

# ----------------- Fixing the final points to generate a correct list of champions, due to points regulation change throughout the years -----------------
team_champ.loc[(team_champ['Year'] == 1958) & (team_champ['Car'] == 'Ferrari'), 'Points'] = 40
team_champ.loc[(team_champ['Year'] == 1958) & (team_champ['Car'] == 'Vanwall'), 'Points'] = 48

team_champ.loc[(team_champ['Year'] == 1972) & (team_champ['Car'] == 'Tyrrell'), 'Points'] = 51
team_champ.loc[(team_champ['Year'] == 1972) & (team_champ['Car'] == 'McLaren'), 'Points'] = 47

team_champ.loc[(team_champ['Year'] == 1974) & (team_champ['Car'] == 'McLaren'), 'Points'] = 73
team_champ.loc[(team_champ['Year'] == 1974) & (team_champ['Car'] == 'Ferrari'), 'Points'] = 65

team_champ.loc[(team_champ['Year'] == 2007) & (team_champ['Car'] == 'McLaren'), 'Points'] = 0

team_champ.loc[(team_champ['Year'] == 1965) & (team_champ['Car'] == 'Lotus'), 'Points'] = 54
team_champ.loc[(team_champ['Year'] == 1965) & (team_champ['Car'] == 'BRM'), 'Points'] = 45

team_champ.loc[(team_champ['Year'] == 1970) & (team_champ['Car'] == 'Lotus'), 'Points'] = 59
team_champ.loc[(team_champ['Year'] == 1970) & (team_champ['Car'] == 'Ferrari'), 'Points'] = 52
team_champ.loc[(team_champ['Year'] == 1970) & (team_champ['Car'] == 'March Ford'), 'Points'] = 48

team_champ.loc[(team_champ['Year'] == 1973) & (team_champ['Car'] == 'Lotus'), 'Points'] = 92
team_champ.loc[(team_champ['Year'] == 1973) & (team_champ['Car'] == 'Tyrrell'), 'Points'] = 82

team_champ.loc[(team_champ['Year'] == 1964) & (team_champ['Car'] == 'Ferrari'), 'Points'] = 45
team_champ.loc[(team_champ['Year'] == 1964) & (team_champ['Car'] == 'BRM'), 'Points'] = 42
team_champ.loc[(team_champ['Year'] == 1964) & (team_champ['Car'] == 'Lotus'), 'Points'] = 37
# ----------------- end of points regulation fix -----------------

# sort the dataframe in descending order by year, then by points
team_champ.sort_values(['Year','Points'], ascending=[False, False], inplace=True) 

# show values
team_champ.groupby(by=['Year'])['Car'].head(1).value_counts()


Ferrari                    14
Mercedes                    8
Williams Renault            5
McLaren Honda               4
Lotus Climax                3
Lotus Ford                  3
Red Bull Racing Renault     3
Williams Ford               2
Cooper Climax               2
Brabham Repco               2
Tyrrell Ford                2
McLaren Ford                2
McLaren TAG                 2
Williams Honda              2
Renault                     2
McLaren Mercedes            2
Benetton Renault            1
Matra Ford                  1
Brawn Mercedes              1
BRM                         1
RBR Renault                 1
Vanwall                     1
Name: Car, dtype: int64

## C) Most titles/total seasons by driver

In [ ]:
# Retrieving number of titles per driver from section A) calculations
champs = driver_champ.groupby(by=['Year'])['Driver'].head(1).value_counts()
champs = champs.reset_index()
champs.columns = ['Driver', 'Titles']

# ----- Calculating seasons participated in by driver -----
# group data by year and driver
seasons = df_race_results.groupby(by=['Year','Driver']).agg({'Driver': 'nunique'})
seasons.columns = ['Seasons']
seasons.reset_index()
# count number of seasons by driver
seasons = seasons.groupby(by=['Driver'])['Seasons'].count()
seasons = seasons.reset_index()

# join tables with number of seasons and number of titles information
ratio = pd.merge(seasons, champs, 'outer', on='Driver')

# calculate title win ratio and sort the data
ratio['Win ratio'] = ratio['Titles']/ratio['Seasons']*100
ratio.sort_values(by='Win ratio', inplace=True, ascending=False)
ratio['Win ratio'] = ratio['Win ratio'].map('{:,.0f}%'.format)

ratio.head(10)

,Driver,Seasons,Titles,Win ratio
431,Juan Manuel Fangio,8,5.0,62%
472,Lewis Hamilton,15,7.0,47%
522,Michael Schumacher,19,7.0,37%
344,Jackie Stewart,9,3.0,33%
14,Alberto Ascari,6,2.0,33%
8,Alain Prost,13,4.0,31%
54,Ayrton Senna,11,3.0,27%
694,Sebastian Vettel,15,4.0,27%
558,Niki Lauda,13,3.0,23%
373,Jim Clark,9,2.0,22%


## D) Most titles/total seasons by team

In [ ]:
# Retrieving number of title per team information from section B) calculations
team_titles = team_champ.groupby(by=['Year'])['Car'].head(1).value_counts()
team_titles = team_titles.reset_index()
team_titles.columns = ['Team', 'Titles']

# ----- Calculating seasons participated in by team -----
df = df_race_results['Car'].unique()
df = pd.DataFrame(df, columns=['Team'])

path = r'/content/drive/MyDrive/Colab Notebooks/F1_dataset/'
df.to_excel(path+'team_names.xlsx', index=False) 

# ----- Calculating seasons participated in by team -----
# group data by year and team
team_seasons = df_race_results.groupby(by=['Year','Car']).agg({'Car': 'nunique'})
team_seasons.columns = ['Seasons']
team_seasons.reset_index()

# count number of seasons by team
team_seasons = team_seasons.groupby(by=['Car'])['Seasons'].count()
team_seasons = team_seasons.reset_index()
team_seasons.columns = ['Team','Seasons']

# join tables with number of seasons and number of titles information
team_ratio = pd.merge(team_seasons, team_titles, 'outer', on='Team')

# calculate title win ratio and sort the data
team_ratio['Win ratio'] = team_ratio['Titles']/team_ratio['Seasons']*100
team_ratio.sort_values(by='Win ratio', inplace=True, ascending=False)
team_ratio['Win ratio'] = team_ratio['Win ratio'].map('{:,.0f}%'.format)

team_ratio.head(10)

,Team,Seasons,Titles,Win ratio
47,Brawn Mercedes,1,1.0,100%
204,Mercedes,12,8.0,67%
248,Red Bull Racing Renault,5,3.0,60%
197,McLaren Honda,8,4.0,50%
45,Brabham Repco,4,2.0,50%
243,RBR Renault,2,1.0,50%
314,Williams Renault,11,5.0,45%
310,Williams Honda,5,2.0,40%
202,McLaren TAG,5,2.0,40%
309,Williams Ford,6,2.0,33%


## E) Most race wins by driver

In [ ]:
# Aggregating race wins information from overalls dataframe
race_winners = df_overalls['Winner'].value_counts()
race_winners.head(10)

Lewis Hamilton        103
Michael Schumacher     91
Sebastian Vettel       53
Alain Prost            51
Ayrton Senna           41
Fernando Alonso        32
Nigel Mansell          31
Jackie Stewart         27
Niki Lauda             25
Jim Clark              25
Name: Winner, dtype: int64

## F) Most race wins by team

In [5]:
# Aggregating race wins information from overalls dataframe
team_race_wins = df_overalls['Car'].value_counts()

# Adjusting Ferrari wins due to 1956 Argentina's GP, where two Ferrari drivers won the race
team_race_wins['Ferrari'] = team_race_wins['Ferrari']-1

team_race_wins.head(10)

Ferrari     238
McLaren     183
Mercedes    124
Williams    114
Lotus        81
Red Bull     75
Renault      35
Brabham      35
Benetton     27
Tyrrell      23
Name: Car, dtype: int64

## G) Most race wins/total races by driver

In [ ]:
# Aggregating race entries information by driver from race results dataframe
total_races_driver = df_race_results['Driver'].value_counts()
total_races_driver = total_races_driver.reset_index()
total_races_driver.columns = ['Driver','Races']

# Retrieving race winners information from section E)
race_winners = df_overalls['Winner'].value_counts()
race_winners = race_winners.reset_index()
race_winners.columns = ['Driver', 'Wins']

# join tables with total races and number of wins information
win_ratio = pd.merge(total_races_driver, race_winners, 'outer', on='Driver')

# calculate race win ratio and sort the data
win_ratio['Win ratio'] = win_ratio['Wins']/win_ratio['Races']*100
win_ratio.sort_values(by='Win ratio', inplace=True, ascending=False)
win_ratio['Win ratio'] = win_ratio['Win ratio'].map('{:,.0f}%'.format)
win_ratio.head(10)

,Driver,Races,Wins,Win ratio
564,Lee Wallard,2,1.0,50%
128,Juan Manuel Fangio,58,24.0,41%
418,Bill Vukovich,5,2.0,40%
184,Alberto Ascari,36,13.0,36%
5,Lewis Hamilton,288,103.0,36%
104,Jim Clark,73,25.0,34%
4,Michael Schumacher,304,91.0,30%
75,Jackie Stewart,99,27.0,27%
37,Ayrton Senna,155,41.0,26%
20,Alain Prost,198,51.0,26%


## H) Most race wins/total races by Team

In [22]:
# Aggregating race entries information by team from race results dataframe
#total_races_team = df_race_results['Car'].value_counts()
#total_races_team = total_races_team.reset_index()
#total_races_team.columns = ['Team','Races']
#total_races_team


# group data by year and team
total_races_team = df_race_results.groupby(by=['Year','race_name','Car']).agg({'Car': 'nunique'})
total_races_team.columns = ['Races']
total_races_team.reset_index(inplace=True)

# count number of seasons by team
total_races_team = total_races_team.groupby(by=['Car'])['Races'].count()
total_races_team = total_races_team.reset_index()
total_races_team.columns = ['Team','Races']

# retrieving race wins by team from section F)
team_race_wins = df_overalls['Car'].value_counts()
team_race_wins = team_race_wins.reset_index()
team_race_wins.columns = ['Team','Wins']
team_race_wins

# join tables with total races and number of wins information
team_win_ratio = pd.merge(total_races_team, team_race_wins, 'outer', on='Team')

# calculate race win ratio and sort the data
team_win_ratio['Win ratio'] = team_win_ratio['Wins']/team_win_ratio['Races']*100
team_win_ratio.sort_values(by='Win ratio', inplace=True, ascending=False)
team_win_ratio['Win ratio'] = team_win_ratio['Win ratio'].map('{:,.0f}%'.format)
team_win_ratio.head(10)

#total_races_team.to_excel('/content/drive/MyDrive/Colab Notebooks/F1_dataset/team_seasons.xlsx')

,Team,Races,Wins,Win ratio
94,Mercedes,249,124.0,50%
21,Brawn,17,8.0,47%
70,Kurtis Kraft,12,5.0,42%
44,Epperly,5,2.0,40%
153,Vanwall,28,10.0,36%
157,Watson,9,3.0,33%
48,Ferrari,1030,239.0,23%
118,Red Bull,326,75.0,23%
92,McLaren,902,183.0,20%
159,Williams,770,114.0,15%


#I) Most podiums by driver

In [24]:
df_race_results.loc[df_race_results['Position'].isin([1,2,3])]

,Position,Driver Number,Driver,Driver Abbreviation,Car,Laps,Time/Retired,Points,Year,race_name
23590,16,6,Nicholas Latifi,LAT,Williams,65.0,+1 lap,0.0,2021,spain
23591,17,14,Fernando Alonso,ALO,Alpine,65.0,+1 lap,0.0,2021,spain
23592,18,47,Mick Schumacher,MSC,Haas,64.0,+2 laps,0.0,2021,spain
23593,19,9,Nikita Mazepin,MAZ,Haas,64.0,+2 laps,0.0,2021,spain
23594,NC,22,Yuki Tsunoda,TSU,AlphaTauri,6.0,DNF,0.0,2021,spain


# 4.2) Who are the best drivers and teams in qualifying pace?


It might be not the case for general F1 fans, but for the hardcore ones it is common sense that the qualifying session and the race are two whole different challanges for both car and driver. While one is focused on taking both to the limit aiming for a flying lap, the other is focused on the endurance/speed couple.

In this section I'll evaluate the metrics listed below:

A) Most poles by driver

B) Most poles by team

C) Most poles/qualys entered by driver

D) Most poles/qualys entered by team

# F1 Ststistics

- most titles by driver
- most titles by team
- most titles/seasons raced by driver
- most titles/seasons raced by team

- most wins by driver
- most wins by team
- most wins/races competed by driver
- most wins/races competed by team


- most podiums by driver
- most podiums by team
- most podiums/races by driver
- most podiums/races by team
--------------------------
- most poles by driver
- most poles by team
- most poles/qualys entered by driver
- most poles/qualys entered by team


- most DNFs by driver
- most DNFs by team
- most DNFs/races competed by driver
- most DNFs/races competed by team

- most home wins by driver

- longest win streak by driver
- longest win streak by team

- longest pole streak by driver
- longest pole streak by team

- longest streak of different winners by driver
- longest streak of different pole by driver

- most distance in points from 1st to 2nd team

- longest steak of races scoring points by driver
- longest steak of races scoring points by team

- longest steak of races NOT scoring points by driver
- longest steak of races NOT scoring points by team

- highest avg points / race by driver
- highest avg points / race by team

# Some numebers

In [ ]:
# Rank the race winners
df_race_winners = df_race_results.loc[df_race_results.Position == '1']
df_race_winners['Driver'].value_counts()


Lewis Hamilton          102
Michael Schumacher       91
Sebastian Vettel         52
Alain Prost              51
Ayrton Senna             41
                       ... 
Jean-Pierre Beltoise      1
Carlos Pace               1
Vittorio Brambilla        1
Jochen Mass               1
Esteban Ocon              1
Name: Driver, Length: 101, dtype: int64

In [ ]:
# Rank the race winners by track
df_race_winners = df_race_results.loc[df_race_results.Position == '1']
df_race_winner_by_track = df_race_winners[['Driver','race_name']].value_counts().reset_index(name='counts')

li =[]

for driver in df_race_winners['Driver'].drop_duplicates():
  dff = df_race_winner_by_track[['race_name','counts']][df_race_winner_by_track['Driver']==driver].T
  dff.columns = dff.iloc[0]
  dff = dff.drop(dff.index[0])
  dff['Driver'] = driver
  li.append(dff)


df_wins_by_track = pd.concat(li,axis=0,ignore_index=True)
#df_wins_by_track.loc[df_wins_by_track['Driver'] == 'Sebastian Vettel']
df_wins_by_track.fillna(0)


race_name,switzerland,italy,great-britain,germany,belgium,Driver,argentina,france,monaco,spain,...,tuscany,styria,saudi-arabia,qatar,emilia-romagna,eifel,azerbaijan,india,70th-anniversary,sakhir
0,1,1,1,1,1,Nino Farina,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2,1,3,3,Juan Manuel Fangio,4,4,2,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,Luigi Fagioli,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,2,2,2,Alberto Ascari,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,Jose Froilan Gonzalez,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0,0,0,0,0,Valtteri Bottas,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
97,0,1,0,0,1,Charles Leclerc,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,Sergio Perez,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
99,0,1,0,0,0,Pierre Gasly,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Rank most DNF by Driver Absolute
df_race_results.loc[df_race_results['Time/Retired'] == 'DNF']['Driver'].value_counts()

# Rank most DNF by Driver Percentual
dnf_driver = df_race_results.loc[df_race_results['Time/Retired'] == 'DNF']['Driver'].value_counts().reset_index(name='counts')
total_races_driver = df_race_results['Driver'].value_counts().reset_index(name='counts')

li =[]

for driver in df_race_results['Driver'].drop_duplicates():
  driver_dnf = pd.to_numeric(dnf_driver.loc[dnf_driver['index'] == driver]['counts'])
  
  driver_total = total_races_driver.loc[total_races_driver['index'] == driver]['counts']
  
  #li.append(np.round(driver_dnf/driver_total,2))
  

# Rank most DNF by Circuit Absolute
#df_race_results.loc[df_race_results['Time/Retired'] == 'DNF']['race_name'].value_counts()
